# Sensors

In [1]:
import os
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

import time
import glob
import tables as tb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import numpy.testing as npt
from operator import itemgetter, attrgetter
%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rcParams["figure.figsize"] = 10, 8
plt.rcParams["font.size"     ] = 14

2017-12-17 15:38:28


In [2]:
from math import sqrt, pi

In [15]:
from pynext.system_of_units import *
from pynext.Sensors import PMT
from pynext.Sensors import SiPM
from pynext.Sensors import KDB

In [4]:
from collections import namedtuple

In [51]:
from pynext.activity_functions import Activity
from pynext.activity_functions import activity_table
from pynext.activity_functions import pmt_activity
from pynext.activity_functions import sipm_activity
from pynext.activity_functions import print_activity

In [6]:
from pynext.CylindricalVessel import CylindricalVessel
from pynext.CylindricalVessel import CVD, CVA
from pynext.NextData import NextPVData
from pynext.Material import vacuum, ti316, cu12, cu03, pb

## NEXT-100 PV

In [7]:
n100d = NextPVData()

In [8]:
cvd_pv = CVD(name    = 'Next100PV',
                 R       = n100d.pv_inner_radius,
                 th_body = n100d.pv_body_thickness,
                 L       = n100d.pv_length,
                 th_head = n100d.pv_head_thickness)

n100_pv = CylindricalVessel(name=cvd_pv.name, material=ti316, cvd=cvd_pv)

In [9]:
n100_pv


        Cylindrical Vessel:

        ----------------
        name      = Next100PV
        material  = 316ti

        specific activity of material:
        Bi-214    =    1.00 mBq/kg
        Tl-208    =    0.15 mBq/kg

        body:
        R         =  680.00 mm
        thickness =   10.00 mm
        length    = 1600.00 mm
        surface   = 6.84e+06 mm2
        volume    = 6.89e+07 mm3
        mass      =  541.96 kg
        activity Bi-214 =  541.96 mBq, self-shielded = 195.19 mBq
        activity Tl-208 =   81.29 mBq, self-shielded =  29.28 mBq
        transmittance   = 7.36e-01

        heads:
        thickness =   12.00 mm
        surface   = 2.91e+06 mm2
        volume    = 3.49e+07 mm3
        mass      =  274.38 kg
        activity Bi-214 =  137.19 mBq, self-shielded =  47.44 mBq
        activity Tl-208 =   20.58 mBq, self-shielded =   7.12 mBq
        transmittance   = 6.92e-01

        

## PMTs

In [10]:
pmt = PMT()
pmt


        PMT name                = R1141
        PMT Diameter            =    7.50 cm
        PMT Surface             =   44.18 cm2
        PMT QE                  =    0.30
        bi214 activity (PMT)    =    0.35 mBq
        tl208 activity (PMT)    =    0.19 mBq
        bi214 activity (Base)   =    1.22 mBq
        tl208 activity (Base)   =    0.33 mBq
        bi214 activity (Window) =    0.61 mBq
        tl208 activity (Window) =    0.38 mBq
        

## SiPMs

In [13]:
sipm = SiPM()
sipm


        SiPM name          = SENSL
        SiPM size          =    1.00 mm
        SiPM Surface       =    1.00 mm2
        SiPM QE            =    0.50
        SiPM TPB eff       =    0.50
        SiPM global PDE    =    0.25
        bi214 activity     =    0.10 muBq
        bi214 activity     =    0.10 muBq
        

## KDBs

In [32]:
kdb = KDB(L = 110  * mm,
          pitch    = 10   * mm,
          nof_sipm = 64,
          a_bi214  = 31 * muBq, a_tl208= 15 * muBq)
kdb


        nof_sipm          = 64
        KDB size          =  110.00 mm
        KDB pitch         =   10.00 mm
        KDB Surface       =  121.00 cm2
        bi214 activity    =   31.00 muBq
        bi214 activity    =   15.00 muBq
        

In [33]:
nof_kdb = n100_pv.head_surface / kdb.S
nof_kdb 

120.0555738032992

In [27]:
nof_sipm = nof_kdb * kdb.nof_sipm
nof_sipm

7683.556723411149

In [44]:
pmtActivity = pmt_activity('PMT activity', 60, pmt)

In [52]:
print_activity('PMT activity', pmtActivity, unit='mBq')


    activity 	 PMT activity
    Bi-214 	  130.80 mBq
    Tl-208 	   54.00 mBq
    


In [46]:
sipmActivity = sipm_activity('SiPM activity', nof_sipm, sipm)

In [53]:
print_activity('SiPM activity', sipmActivity, unit='mBq')


    activity 	 SiPM activity
    Bi-214 	    0.77 mBq
    Tl-208 	    0.77 mBq
    


In [54]:
kdbActivity = sipm_activity('KDB activity', nof_kdb , kdb)
print_activity('KDB activity', kdbActivity, unit='mBq')


    activity 	 KDB activity
    Bi-214 	    3.72 mBq
    Tl-208 	    1.80 mBq
    


In [50]:
act_sensor_table =activity_table([pmtActivity, sipmActivity, kdbActivity])
print(act_sensor_table)

            name       bi214      tl208
0   PMT activity  130.800000  54.000000
1  SiPM activity    0.768356   0.768356
2   KDB activity    3.721723   1.800834


In [55]:
act_sensor_table

,name,bi214,tl208
0,PMT activity,130.800000,54.000000
1,SiPM activity,0.768356,0.768356
2,KDB activity,3.721723,1.800834
